In [ ]:
pip install trjtrypy

In [ ]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from termcolor import colored
import os
import pickle

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# install package

In [ ]:
# to make a directory
%cd 'gdrive/My Drive/traj-dist'

/content/gdrive/My Drive/traj-dist


In [ ]:
# to see in what directory we are in

In [ ]:
pwd

'/content/gdrive/My Drive/traj-dist'

In [ ]:
# to install setup.py from the current directory
!python setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/lib/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'download_urt'
  warnings.warn(msg)
running install
running bdist_egg
running egg_info
writing traj_dist.egg-info/PKG-INFO
writing dependency_links to traj_dist.egg-info/dependency_links.txt
writing requirements to traj_dist.egg-info/requires.txt
writing top-level names to traj_dist.egg-info/top_level.txt
adding license file 'LICENSE.txt'
writing manifest file 'traj_dist.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
skipping 'traj_dist/cydist/basic_geographical.c' Cython extension (up-to-date)
skipping 'traj_dist/cydist/basic_euclidean.c' Cython extension (up-to-date)
skipping

In [ ]:
!pip install geohash2

# "######################" Now restart runtime "######################"

But don't run the above block

In [ ]:
import traj_dist.distance as tdist
import pickle

# Import data

In [ ]:
I = glob.glob('/content/gdrive/My Drive/Research/Beijing for 2ed paper/Chosen users for 4 pairs/**/*.plt',
              recursive=True)
S = list(set([I[i][:84] for i in range(len(I))]))
user_idx = np.array([S[i][81:] for i in range(len(S))])
print(user_idx)
len(S), S[0]

['044' '040' '016' '015' '033' '125']


(6,
 '/content/gdrive/My Drive/Research/Beijing for 2ed paper/Chosen users for 4 pairs/044')

In [ ]:
def read_file(path):
    J = glob.glob(path, recursive=True)
    data = [0] * len(J)
    c = 0
    for j in range(len(J)):
        data[j] = []
        with open(J[j], "r") as f:
            for line in f:
                if c > 6:
                    item = line.strip().split(",")
                    if len(item) == 7:
                        data[j].append(np.array([float(item[0]), float(item[1]), 
                                                 float(item[4])]))
                c += 1
        data[j] = np.array(data[j])
    return np.array(data)

### data1[i] in the following is user_i from Beijing dataset 

trajectories have time dimension in data1

In [ ]:
users = np.array([15, 16, 33, 40, 44, 125])

In [ ]:
start_time = time.time()
user_idx_int = np.array(list(map(int, user_idx)))
data1 = []
for i in [15, 16, 33, 40, 44, 125]:
    idx = np.where(user_idx_int == i)[0][0]
    path = S[idx]+'/**/*.plt'
    J = glob.glob(path, recursive=True)
    data1.append(read_file(path))
data1 = np.array(data1)
print(time.time() - start_time)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


56.63820791244507


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [ ]:
[len(data1[i]) for i in range(len(data1))] # = [67, 51, 13, 27, 72, 57]

[67, 51, 13, 27, 72, 57]

In [ ]:
data_1 = data1 + 0
len(data_1)

6

In [ ]:
def remove_segments(traj): # removes stationary points
    p2 = traj[:,:2][1:]
    p1 = traj[:,:2][:-1]
    L = ((p2-p1)*(p2-p1)).sum(axis =1)
    I = np.where(L>1e-16)[0]
    return traj[I]

### In data_1 below:
    (1) there is no stationary points
    (2) there is no trajectories with more than 200 waypoints

In [ ]:
for i in range(len(data_1)):
    data_1[i] = np.array(list(map(remove_segments, data_1[i])), dtype='object')
    L = np.array([len(data_1[i][j]) for j in range(len(data_1[i]))])
    I = np.where((L > 1000))[0]
    data_1[i] = data_1[i][I]

I = np.where(np.array([len(data_1[i]) for i in range(len(data_1))]) > 0)[0]
data_1 = data_1[I]
print("len(data_1) =", len(data_1))
print("selected users:", users[I]) # now 0,1,2,3,4,5 

len(data_1) = 6
selected users: [ 15  16  33  40  44 125]


# Partitioning trajectories to less than 20 minutes long

In [ ]:
def partition(trajectory, threshold=20):
    '''threshold is in minutes'''
    trajectories = []
    a = 24 * 60 * sum(trajectory[:,2][1:] - trajectory[:,2][:-1])
    if a <= threshold:
        return np.array(trajectory.reshape(1, len(trajectory), 3))
    else: 
        i = 0
        while a > threshold:
            j = i + 0
            val = 0
            while val < threshold: 
                if i < len(trajectory) - 1:
                    temp = val + 0
                    val += 24 * 60 * (trajectory[:,2][1:][i] - trajectory[:,2][:-1][i])
                    i += 1
                else: 
                    break
            if len(trajectory[j:i-1]) > 0:
                trajectories.append(trajectory[j:i-1])
            a = a - val
        if len(trajectory[i:]) > 0:
            trajectories.append(trajectory[i:])
    return np.array(trajectories)

In [ ]:
# 24 * 60 * (days_date('1899/12/30 2:50:06') - days_date('1899/12/30 2:20:06')) == 20 min
Time = [0] * len(data_1)
for i in range(len(data_1)):
    Time[i] = []
    for j in range(len(data_1[i])):
        Time[i].append(24 * 60 * sum(data_1[i][j][:,2][1:] - data_1[i][j][:,2][:-1])) # = 20 minutes 
    Time[i] = np.array(Time[i], dtype='object')

Time = np.array(Time, dtype='object')
Time.shape

(6,)

In [ ]:
J = [np.where(Time[i] > 20)[0] for i in range(len(Time))]
print(len(J))

6


### data3 below is the array of trajectories having less than 20 minutes long

In [ ]:
data3 = [0] * len(data_1)

for i in range(len(data_1)):
    data3[i] = []
    for j in range(len(data_1[i])):
        A = partition(data_1[i][j], threshold=20)
        for k in range(len(A)):
            data3[i].append(A[k])
    data3[i] = np.array(data3[i], dtype='object')
    
data3 = np.array(data3, dtype='object')

data3.shape, data3[0].shape, data3[0][0].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


((6,), (302,), (150, 3))

In [ ]:
data4 = data3 + 0

data4 is the users having between 100 and 200 trajectories and each has length between 10 and 200 trajectory

In [ ]:
for i in range(len(data4)):
    A = np.array([len(data4[i][j]) for j in range(len(data4[i]))])
    I = np.where((A > 10) & (A < 200))[0]
    data4[i] = data4[i][I]
    
print(len(data4))
A = np.array([len(data4[i]) for i in range(len(data4))])
print(A)
chosen_users = np.where((A > 100) & (A < 200))[0]
data4 = data4[chosen_users]

print("chosen users:", chosen_users)
print("len(data4) =", len(data4))
A = [len(data4[i]) for i in range(len(data4))]
print("length of preprocessed users in data4:", np.sort(A))

6
[154 140 117 193 197 122]
chosen users: [0 1 2 3 4 5]
len(data4) = 6
length of preprocessed users in data4: [117 122 140 154 193 197]


### data2 is the same as data4 but without time dimension

In [ ]:
data = data4 + 0
for i in range(len(data)):
    data[i] = np.array([data4[i][j][:,:2] for j in range(len(data4[i]))], dtype='object')
len(data)

6

In [ ]:
[len(data[i]) for i in range(len(data))] # = [154, 140, 117, 193, 197, 122]

[154, 140, 117, 193, 197, 122]

In [ ]:
pairs =  [[0, 4], [0, 5], [1, 4], [2, 3]] # equivalent to [[4, 12], [4, 16], [5, 12], [8, 10]] in Anaconda

In [ ]:
def get_sigma(train_1, train_2):
    a = np.mean([np.mean(train_1[i], 0) for i in range(len(train_1))], 0)
    b = np.mean([np.mean(train_2[i], 0) for i in range(len(train_2))], 0)
    return np.round(max(abs(a-b)), decimals=4)

In [ ]:
for i in range(4):
    m, n = pairs[i]
    print(get_sigma(data[m], data[n])) # r = 0.01,1,0.1,0.1

0.0086
2.0693
0.0424
0.1256


In [ ]:
0.0086/0.01, 2.0693, 0.0424/0.1, 0.1256/0.1

(0.86, 2.0693, 0.424, 1.2559999999999998)

In [ ]:
# waypoints max length calculator 
def length_max(x):
    p1 = x[:-1]
    p2 = x[1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    return max(L)

In [ ]:
def get_sigma_mean_max(data1, data2):
    Mean_waypoints_1 = np.mean([length_max(data1[i]) for i in range(len(data1))])
    Mean_waypoints_2 = np.mean([length_max(data2[i]) for i in range(len(data2))])
    Mean1 = np.round(Mean_waypoints_1, decimals=3) 
    Mean2 = np.round(Mean_waypoints_2, decimals=3)
    Mean = np.round((Mean_waypoints_1 + Mean_waypoints_2)/2, decimals=3)

    Max_waypoints_1 = np.max([length_max(data1[i]) for i in range(len(data1))])
    Max_waypoints_2 = np.max([length_max(data2[i]) for i in range(len(data2))])
    Max1 = np.round(Max_waypoints_1, decimals=3) 
    Max2 = np.round(Max_waypoints_2, decimals=3)
    Max = np.round((Max_waypoints_1 + Max_waypoints_2)/2, decimals=3)

    return Mean1, Mean2, Max1, Max2, Mean, Max

In [ ]:
for i in range(4):
    m, n = pairs[i]
    print(get_sigma_mean_max(data[m], data[n])[-2:]) # r = 0.01,1,0.1,0.1

(0.003, 0.051)
(0.004, 0.114)
(0.003, 0.05)
(0.001, 0.018)


# From github page: https://github.com/bguillouet/traj-dist

## Generating distance matrices (Continuous Frechet)

In [ ]:
pairs

[[0, 4], [0, 5], [1, 4], [2, 3]]

In [ ]:
# data = np.concatenate((data1, data2), 0)
def calculate_dists_line_by_line_frechet(data, pair_idx, start_idx, end_idx): 
    n = len(data)
    for i in range(start_idx, end_idx):
        start_time = time.time()
        def f(traj):
            return tdist.frechet(data[i], traj)
        A = list(map(f, data[i+1:]))
        np.savetxt('/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (Beijing)/frechet lines '+str(pair_idx)+'/'+str(i)+'.csv', 
                   A, delimiter=',')
        total_time = time.time() - start_time
        print(f"time for step {i}: {total_time}")
    return total_time

In [ ]:
#j = 0,1,2,3
Data = np.concatenate((data[pairs[j][0]], data[pairs[j][1]]), 0)
calculate_dists_line_by_line_frechet(Data, pair_idx=j, start_idx=0, end_idx=len(Data))

## To stack the 1d generated vectors together to get the distance matrix of continuous Frechet distance we will use the following get_matrix function.

In [ ]:
def get_matrix(data1, data2, pair_idx, path):
    '''
    path is: 
    '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (Beijing)/frechet lines '
    '''
    n = len(data1) + len(data2)
    matrix = np.zeros((n,n))
    for i in range(n-1):
        #matrix[i][i+1:] = np.array(pd.read_csv(path+str(i)+'.csv', header=None)).reshape(n-i-1,)
        matrix[i][i+1:] = np.loadtxt(path+str(pair_idx)+'/'+str(i)+'.csv')
    for i in range(1, n):
        for j in range(i):
            matrix[i][j] = matrix[j][i]
    np.savetxt(path[:-14]+'frechet '+str(pair_idx)+'.csv', matrix, delimiter=',')
    return matrix

# KNN with Continuous Frechet

In [ ]:
def KNN_with_dists(n_1, n_2, dists_names, paths_to_dists):
    '''path example: '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/sspd.csv'
       dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)
       n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    train_errors = np.zeros(len(dists_names))
    test_errors = np.zeros(len(dists_names))

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    for i in range(len(dists_names)):

        dist_matrix = np.array(pd.read_csv(paths_to_dists[i], header=None))

        D_train = dist_matrix[I][:, I]
        D_test = dist_matrix[J][:,I]
        train_labels = labels[I]
        test_labels = labels[J]

        clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
        
        #Train the model using the training sets
        clf.fit(D_train, list(train_labels))

        #Predict labels for train dataset
        train_pred = clf.predict(D_train)
        train_errors[i] = sum(train_labels != train_pred)/len(I)
        
        #Predict labels for test dataset
        test_pred = clf.predict(D_test)
        test_errors[i] = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [ ]:
def KNN_average_error(data1, data2, num_trials, dists_names, paths_to_dists):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros((num_trials, len(dists_names)))
    test_errors = np.zeros((num_trials, len(dists_names)))

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_dists(len(data1), len(data2), dists_names, paths_to_dists)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors, axis=0)
    test_error = np.mean(test_errors, axis=0)
    std_test_error = np.std(test_errors, axis=0)

    Dict = {}
    for i in range(len(dists_names)):
        Dict[i+1] = [f"KNN with {dists_names[i]}", 
                     np.round(train_error[i], decimals = 4), 
                     np.round(test_error[i], decimals = 4), 
                     np.round(std_test_error[i], decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, train_errors, test_errors, train_error, test_error)

In [ ]:
path = '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (Beijing)/frechet lines '

In [ ]:
for j in range(len(pairs)):
    A = get_matrix(data[pairs[j][0]], data[pairs[j][1]], pair_idx=j, path=path)
    #print(A.shape, len(data[pairs[j][0]])+len(data[pairs[j][1]]))

In [ ]:
test_error = []
test_std = []
for j in range(len(pairs)):
    F = KNN_average_error(data[pairs[j][0]], data[pairs[j][1]], num_trials=50, 
                        dists_names=['frechet'], 
                        paths_to_dists=[path[:-14]+'frechet '+str(j)+'.csv'])
    print(colored(pairs[j], 'yellow'))
    print('pair:', F[0])
    print(colored('==========================================================', 'red'))
    test_error.append(np.array(F[0])[0][2])
    test_std.append(np.array(F[0])[0][3])
print(colored(f'mean test error: {np.mean(test_error)}', 'magenta'))
print(colored(f'mean std of error: {np.mean(test_std)}', 'blue'))

num_trials = 50
total time = 3.8871326446533203
[0, 4]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.2258      0.2714  0.0339
num_trials = 50
total time = 2.616995096206665
[0, 5]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.1397      0.1967  0.0432
num_trials = 50
total time = 3.426722764968872
[1, 4]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.1813      0.2547  0.0355
num_trials = 50
total time = 3.0371618270874023
[2, 3]
pair:          Classifier  Train Error  Test Error     std
1  KNN with frechet       0.1372       0.217  0.0364
mean test error: 0.23494999999999996
mean std of error: 0.037250000000000005


# KNN with LSH

In [ ]:
pairs

[[0, 4], [0, 5], [1, 4], [2, 3]]

In [ ]:
#pip install trjtrypy

In [ ]:
from google.colab import files
files.upload()

import KNN_with_LSH_class
from KNN_with_LSH_class import KNN_with_LSH

Saving KNN_with_LSH_class.py to KNN_with_LSH_class.py


In [ ]:
test_error_LSH = []
test_std_LSH = []
for j in range(len(pairs)):
    m, n = pairs[j]
    print(len(data[m])+len(data[n]))
    KNN_with_LSH_class = KNN_with_LSH(data[m], data[n], number_circles=20, num_trials=50)
    F = KNN_with_LSH_class.KNN_LSH_average_error()
    print(colored(pairs[j], 'yellow'))
    print('pair:', F[0])
    print(colored('==========================================================', 'red'))
    test_error_LSH.append(np.array(F[0])[0][2])
    test_std_LSH.append(np.array(F[0])[0][3])
print(colored(f'mean test error: {np.mean(test_error_LSH)}', 'magenta'))
print(colored(f'mean std of error: {np.mean(test_std_LSH)}', 'blue'))

351
num_trials = 50
total time = 12.805943965911865
[0, 4]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.3351      0.3806  0.0621
276
num_trials = 50
total time = 9.482008934020996
[0, 5]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.3977      0.3983  0.0827
337
num_trials = 50
total time = 12.003517150878906
[1, 4]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.3809      0.4124  0.0766
310
num_trials = 50
total time = 11.570326566696167
[2, 3]
pair:      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.2817      0.2915  0.0974
mean test error: 0.37070000000000003
mean std of error: 0.0797
